In [1]:
# Imports
!pip install sympy --quiet
from scipy.integrate import solve_bvp
import numpy as np
import scipy.optimize as opt       # import root-finding algorithm
import sympy as sp                 # Python toolbox for symbolic maths
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D # Toolbox for rendring 3D figures
from mpl_toolkits import mplot3d   # Toolbox for rendring 3D figures
# @title Figure Settings
import ipywidgets as widgets  # interactive display
from ipywidgets import interact
%config InlineBackend.figure_format = 'retina'
# use NMA plot style
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/nma.mplstyle")
my_layout = widgets.Layout()

fig_w, fig_h = 12, 4.5
my_fontsize = 16
my_params = {'axes.labelsize': my_fontsize,
          'axes.titlesize': my_fontsize,
          'figure.figsize': [fig_w, fig_h],
          'font.size': my_fontsize,
          'legend.fontsize': my_fontsize-4,
          'lines.markersize': 8.,
          'lines.linewidth': 2.,
          'xtick.labelsize': my_fontsize-2,
          'ytick.labelsize': my_fontsize-2}

plt.rcParams.update(my_params)

In [2]:
def PlotSingleSimSingleProtein(x,y1,y2,lab1,lab2,xlab,ylab,title_string,file_name,width=10,height=8,fsize=16,save_it = 1):
    print(width,height)
    fig, axes = plt.subplots(1, 2,figsize=(width, height))
    ax = axes.ravel()
#     plt.rc('font', **{'family':'serif','serif':['Palatino']})
    plt.rc('text', usetex=False)
    ax[0].plot(x,y1,label=lab1)
    ax[0].set_xlabel(xlab,fontsize=fsize)
    ax[0].set_ylabel(ylab,fontsize=fsize)
    plt.title(title_string,fontsize=fsize)
    ax[1].plot(x,y2,label=lab2)
    ax[1].set_xlabel(xlab,fontsize=fsize)
    ax[1].set_ylabel(ylab,fontsize=fsize)
    plt.title(title_string,fontsize=fsize)
    plt.legend(prop={'size': fsize})
    # plt.show()
    folder = "."
    if save_it == 1:
        plt.savefig("%s/%s.%s"%(folder,file_name,"png"),dpi=150)
        plt.savefig("%s/%s.%s"%(folder,file_name,"eps"),dpi=150)
        print("saved figures to: {%s/%s}" %(folder, file_name))
    else:
        print("Plots not saved")
    plt.show()
    

In [3]:
def PlotSingleSimTwoProtein(x,ps,pc,lab_ps,lab_pc,xlab,ylab,title_string,file_name,width=10,height=8,fsize=16,save_it = 1):
    fig, ax = plt.subplots(figsize=(width, height))
#     plt.rc('font', **{'family':'serif','serif':['Palatino']})
#     plt.rc('text', usetex=True)
    ax.plot(x,ps,label=lab_ps,color=color_list[0])
    ax.plot(x,pc,label=lab_pc,color=color_list[1])
    ax.set_xlabel(xlab,fontsize=fsize)
    ax.set_ylabel(ylab,fontsize=fsize)
    plt.title(title_string,fontsize=fsize)
    plt.legend(prop={'size': fsize})
    # plt.show()
    folder = "."
    if save_it == 1:
        plt.savefig("%s/%s.%s"%(folder,file_name,"png"),dpi=150)
        plt.savefig("%s/%s.%s"%(folder,file_name,"eps"),dpi=150)
        print("saved figures to: {%s/%s}" %(folder, file_name))
    else:
        print("Plots not saved")
    plt.show()
    

In [4]:
L = 221     #length of the dendrite

## 1. One protein with constant velocity and max synaptic uptake
## $ \frac{\partial P}{\partial t} = D_p \frac{\partial^2 P}{\partial x^2} - V_p\frac{\partial P}{\partial x} - \lambda_p P - \eta_{pmax}tanh(\eta_{p0}P) $

In [5]:
class SingleProteinModelWihtCappedSpinesUptakeConstantV():
    def __init__(self,D_p,V_p,half_life,Jin,eta_p_max,eta_p_zero):
        self.D_p = D_p   # in uM^2/s
        self.V_p = V_p    # in uM/s
        self.half_life =half_life # in days
        self.Lamda_p = np.log(2)/(self.half_life*24*60*60);
        self.Jin = Jin;
        self.eta_p_max = eta_p_max;
        self.eta_p_zero = eta_p_zero
        
    def updateModelParams(self,D_p=None,V_p=None,half_life = None,Jin = None,eta_p_max=None,eta_p_zero=None):
        if D_p:
            self.D_p = D_p
        if V_p:
            self.V_p = V_p
        if half_life:
            self.half_life = half_life
            self.Lamda_p = np.log(2)/(self.half_life*24*60*60);
        if Jin:
            self.Jin = Jin
        if eta_p_max:
            self.eta_p_max = eta_p_max
        if eta_p_zero:
            self.eta_p_zero = eta_p_zero
    
    def fun(self,x,y):
        # print(L)
        p,dp = y
        return dp, (self.Lamda_p/self.D_p)*p + (self.eta_p_max/self.D_p)*np.tanh(self.eta_p_zero*p) + (self.V_p/self.D_p)*dp
        
    def bc(self,ya,yb):
        return np.array([self.D_p*ya[1] - self.V_p*ya[0] + self.Jin, \
                         self.D_p*yb[1] - self.V_p*yb[0]])

    def solveModel(self):
        delta_x = 0.01; #step size for the numerical simulation
        # print(len(x_line))
        # solving model
        # D_p * p'' - (V_p(x)*p)' - Lamda_p*p = 0
        x=np.arange(0,L,delta_x)
        # print(x)
        # params=np.array([L]);
        y = np.zeros((2,x.size))
        soln = solve_bvp(self.fun, self.bc, x, y,tol=1e-12,bc_tol=1-12,max_nodes=1e+12,verbose=2)
        print(soln.__dict__)
        print("***************************")
        print(np.shape(soln))
        p_dist = soln.sol(x)[0]
        diff_p_dist = soln.sol(x)[1]
        res = soln.rms_residuals
        print(np.max(res))
#         erros1,erros2 = self.fun(x,[p_dist,diff_p_dist])
        # norm_p_dist = p_dist/p_dist[0]
        # print(len(p_dist))
        self.IntegralBC(delta_x,p_dist,diff_p_dist)
        return x,p_dist
     
    def IntegralBC(self,delta_x,p,diff_p):
        print("***************************")
#         total_p = self.Jin/(self.Lamda_p)
        # x,p= self.solveModel()
        shouldbe_influx = self.D_p*diff_p[0] - self.V_p*p[0]
        lost_deg = self.Lamda_p*np.sum(p)*delta_x;
        lost_spine  = np.sum(self.eta_p_max*np.tanh(self.eta_p_zero*p))*delta_x
        num_total_p = -self.Jin + lost_deg + lost_spine 
        print("influx numerical =",shouldbe_influx)
        print("lost to degradation",lost_deg,"lost to spines",lost_spine,"protein lost/found  in the void = ",num_total_p)
    

In [7]:
def RunSim(v=0.1,Jin=4000,eta_max=60):
    sim_id = "001";
    SP_model1 = SingleProteinModelWihtCappedSpinesUptakeConstantV(0.45,v,20,Jin,eta_max,0.37);
    x,p_dist = SP_model1.solveModel()
    title_string = "test";#"Steady-state spatial distribution \n parameters: $D_p = {%.2f}, V_p = {%.1e}, \lambda_p = {%.2e}, Jin= %.2f, \eta_{pmax} =%.1e,\eta_{p0}$ =%.1e" \
       # %( SP_model1.D_p, SP_model1.V_p, SP_model1.Lamda_p,SP_model1.Jin,SP_model1.eta_p_max,SP_model1.eta_p_zero);
    print("v =%.5f \nJin = %2d \neta_max = %.2e \neta_0 = %.2e"%(v,Jin,eta_max,eta_zero),end=",")
    lab =  'AMPA-R'
    x_label = 'Dendritic distance in (muM)';
    y_label= 'Protein number';
    folder= "Figures/OneProtein/ConstantV/WithCappedUptake/";
    file_name = folder+"SingleProtein_SingleSim__CappedSpineUptake{0}".format(sim_id);
    # pwa = PlottingWidgetAMPA.PlottingWidgetAMPA()
    # pwa.CreateFolderRecursive(folder)
    # PlotSingleSimSingleProtein(x, p_dist, lab, x_label, y_label, title_string, file_name,fsize=14,save_it = 0)

    title_string = "test spine";#"Steady-state spatial distribution in spines \n parameters: $D_p = {%.2f}, V_p = {%.1e}, \lambda_p = {%.2e}, Jin= %.2f, \eta_{pmax} =%.1e,\eta_{p0}$ =%.1e" \
    # %( SP_model1.D_p, SP_model1.V_p, SP_model1.Lamda_p,SP_model1.Jin,SP_model1.eta_p_max,SP_model1.eta_p_zero);
    lab_p_spine =  'Spine-AMPA-R'
    # x_label = 'Dendritic distance in (muM)';
    # y2_label= 'Protein number';
    file_name = folder+"Spine_SingleSim_OneProtein_CappedUptake_dist_{0}".format(sim_id);
    p_spine = SP_model1.eta_p_max*np.tanh(SP_model1.eta_p_zero*p_dist);
    PlotSingleSimSingleProtein(x, p_dist,p_spine, lab,lab_p_spine, x_label, y_label, title_string, file_name,fsize=14,save_it = 0)

_ = widgets.interact(RunSim,v = (0,1,0.05), Jin = (4000,20000,2000), eta_max = (10,60,5))

interactive(children=(FloatSlider(value=0.1, description='v', max=1.0, step=0.05), IntSlider(value=4000, descr…

## 2. One protein with constant velocity and synaptic uptake

## $ \frac{\partial P}{\partial t} = D_p \frac{\partial^2 P}{\partial x^2} - V_p\frac{\partial P}{\partial x} - \lambda_p P - \eta_{p}P $

In [10]:
class SingleProteinModelWihtSpinesConstantV():
    def __init__(self,D_p,V_p,half_life,Jin,eta_p):
        self.D_p = D_p   # in uM^2/s
        self.V_p = V_p    # in uM/s
        self.half_life =half_life # in days
        self.Lamda_p = np.log(2)/(self.half_life*24*60*60);
        self.Jin = Jin;
        self.eta_p = eta_p;
        # self.iota = iota
    def updateModelParams(self,D_p=None,V_p=None,half_life = None,Jin = None,eta_p=None):
        if D_p:
            self.D_p = D_p
        if V_p:
            self.V_p = V_p
        if half_life:
            self.half_life = half_life
            self.Lamda_p = np.log(2)/(self.half_life*24*60*60);
        if Jin:
            self.Jin = Jin
        if eta_p:
            self.eta_p = eta_p
    
    def fun(self,x,y):
        # print(L)
        return y[1], ((self.Lamda_p+self.eta_p)/self.D_p)*y[0] + (self.V_p/self.D_p)*y[1]
        
    def bc(self,ya,yb):
        return np.array([self.D_p*ya[1] - self.V_p*ya[0] + self.Jin, self.D_p*yb[1] - self.V_p*yb[0]])

    def solveModel(self):
        delta_x = 0.0114; #step size for the numerical simulation
        # print(len(x_line))
        # solving model
        # D_p * p'' - (V_p(x)*p)' - Lamda_p*p = 0
        x=np.arange(0,L,delta_x)
        # print(x)
        # params=np.array([L]);
        y = np.zeros((2,x.size))
        soln = solve_bvp(self.fun, self.bc, x, y,max_nodes=1e+12,verbose=2)
        # print(len(soln))
        p_dist = soln.sol(x)[0]
        # norm_p_dist = p_dist/p_dist[0]
        # print(len(p_dist))
        res = soln.rms_residuals
        print("max residual = ",np.max(res))
        self.IntegralBC(delta_x, p_dist)
        return x,p_dist

    def ParameterEffect(self,scales,D_p=None,V_p = None,half_life = None,Jin = None,eta_p=None):
        output = {};
        for ind,scale in enumerate(scales):
            current_D_p = self.D_p
            current_V_p = self.V_p 
            current_half_life = self.half_life
            current_Jin = self.Jin
            current_eta_p = self.eta_p
            # output[ind] = SingleProteinModelWithoutSpines(self.D_p,self.V_p,self.half_life,self.Jin)
            if D_p:
                current_D_p *= scale
            if V_p:
                current_V_p *= scale
            if half_life:
                current_half_life *= scale
            if Jin:
                current_Jin *= scale
            if eta_p:
                current_eta_p *= scale
            output[ind] = SingleProteinModelWihtSpinesConstantV(current_D_p,current_V_p,current_half_life,current_Jin,current_eta_p)
            print(output[ind].__dict__)
        return output

    def IntegralBC(self,delta_x,p):
#         total_p = self.Jin/(self.Lamda_p+self.eta_p);
        # x,p= self.solveModel()
        num_total_p = -self.Jin+((self.Lamda_p+self.eta_p)*np.sum(p)*delta_x)
#         print("total proteins = ",total_p)
        print("protein lost/found  in the void = ",num_total_p)
    

In [11]:
def RunSim2(v,Jin=4000,eta_zero=1/2.67):
    sim_id = "001"
    SP_model1 = SingleProteinModelWihtSpinesConstantV(0.45,v,20,Jin,eta_zero);
    x,p_dist = SP_model1.solveModel()
    print("v =%.5f \nJin = %2d \neta_0 = %.2e"%(v,Jin,eta_zero),end=",")
    # SP_model1.IntegralBC(x,p_dist)
    # SP_model1.updateModelParams(V_p=0.001);
    # SP_model1.solveModel()
    title_string = "test spines";
#     title_string = "Steady-state spatial distribution \n parameters: $D_p = {%.2f}, V_p = {%.1e}$, half-life = %.2f, Jin= %.2f, $\eta_p$ =%.1e" \
#         %( SP_model1.D_p, SP_model1.V_p, SP_model1.half_life,SP_model1.Jin,SP_model1.eta_p);
    lab =  'AMPA-R'
    x_label = r'Dendritic distance (in $\mu$M)';
    y_label= r'Protein number';
    folder= "Figures/OneProtein/ConstantV/WithUptake/";
    file_name = folder+"SingleProtein_SingleSim_{0}".format(sim_id);
    
#     pwa = PlottingWidgetAMPA()
#     pwa.CreateFolderRecursive(folder)
#     PlotSingleSimSingleProtein(x, p_dist, lab, x_label, y_label, title_string, file_name)
    
#     title_string = "Steady-state spatial distribution in Spines\n parameters: $D_p = {%.2f}, V_p = {%.1e}$, half-life = %.2f, Jin= %.2f, $\eta_p$ =%.1e" \
#     %( SP_model1.D_p, SP_model1.V_p, SP_model1.half_life,SP_model1.Jin,SP_model1.eta_p);
    lab_p_spine =  'Spine-AMPA-R'
    x_label = r'Dendritic distance (in $\mu$M)';
    y_label= r'Protein number';
    # folder= "Figures/OneProtein/WithUptake/";
    file_name = folder+"Spine_SingleProtein_SingleSim_{0}".format(sim_id);
    p_spine = SP_model1.eta_p*p_dist;
    
    PlotSingleSimSingleProtein(x, p_dist/p_dist[0],p_spine/p_dist[0], lab,lab_p_spine, x_label, y_label, title_string, file_name,fsize=14,save_it = 0)

_ = widgets.interact(RunSim2,v = (0.00001,0.001,0.00001), Jin = (4000,40000,10000), eta_zero = (0, 1.5, .05))

interactive(children=(FloatSlider(value=0.0005000000000000001, description='v', max=0.001, min=1e-05, step=1e-…

## 3. One protein with linear velocity decrease and capped synaptic uptake

## $ \frac{\partial P}{\partial t} = D_p \frac{\partial^2 P}{\partial x^2} - \frac{\partial (V_p(x) P}{\partial x} - \lambda_p P - \eta_{pmax} tanh(\eta_{p0} P) $

In [12]:
class SingleProteinModelWihtCappedSpinesUptake():
    def __init__(self,D_p,V_p,half_life,Jin,eta_p_max,eta_p_zero):
        self.D_p = D_p   # in uM^2/s
        self.V_p = V_p    # in uM/s
        self.half_life =half_life # in days
        self.Lamda_p = np.log(2)/(self.half_life*24*60*60);
        self.Jin = Jin;
        self.eta_p_max = eta_p_max;
        self.eta_p_zero = eta_p_zero
        
    def updateModelParams(self,D_p=None,V_p=None,half_life = None,Jin = None,eta_p_max=None,eta_p_zero=None):
        if D_p:
            self.D_p = D_p
        if V_p:
            self.V_p = V_p
        if half_life:
            self.half_life = half_life
            self.Lamda_p = np.log(2)/(self.half_life*24*60*60);
        if Jin:
            self.Jin = Jin
        if eta_p_max:
            self.eta_p_max = eta_p_max
        if eta_p_zero:
            self.eta_p_zero = eta_p_zero
    
    def fun(self,x,y):
        # print(L)
        return y[1], ((self.Lamda_p - self.V_p/L)/self.D_p)*y[0] + (self.eta_p_max/self.D_p)*np.tanh(self.eta_p_zero*y[0])+ ((self.V_p*(1-x/L))/self.D_p)*y[1]
        
    def bc(self,ya,yb):
        return np.array([self.D_p*ya[1] - self.V_p*ya[0] + self.Jin, self.D_p*yb[1]])

    def solveModel(self):
        delta_x = 0.0114; #step size for the numerical simulation
        # print(len(x_line))
        # solving model
        # D_p * p'' - (V_p(x)*p)' - Lamda_p*p = 0
        x=np.arange(0,L,delta_x)
        # print(x)
        # params=np.array([L]);
        y = np.zeros((2,x.size))
        soln = solve_bvp(self.fun, self.bc, x, y,max_nodes=1e+12,verbose=2)
        # print(len(soln))
        p_dist = soln.sol(x)[0]
        # norm_p_dist = p_dist/p_dist[0]
        # print(len(p_dist))
        self.IntegralBC(delta_x,p_dist)
        return x,p_dist
  
    def IntegralBC(self,delta_x,p):
        num_total_p = -self.Jin+(self.Lamda_p*np.sum(p) + np.sum(self.eta_p_max*np.tanh(self.eta_p_zero*p)))*delta_x
#         print("total proteins = ",total_p)
        print("protein lost/found  in the void = ",num_total_p)
    

In [14]:
def RunSim3(v=0.1,Jin=4000,eta_max=60,eta_zero=0.37):
    print("v =%.5f \n Jin = %2d \n eta_max = %.2e \n eta_0 = %.2e \n"%(v,Jin,eta_max,eta_zero),end="")
    sim_id = "002";
    SP_model1 = SingleProteinModelWihtCappedSpinesUptake(0.45,v,20,Jin,eta_max,eta_zero);
    x,p_dist = SP_model1.solveModel()
#     title_string = "Steady-state spatial distribution \n parameters: $D_p = {%.2f}, V_p = {%.1e}$, half-life = %.2f, Jin= %.2f, $\eta_{pmax}$ =%.1e,$\eta_{p0}$ =%.1e" \
#         %( SP_model1.D_p, SP_model1.V_p, SP_model1.half_life,SP_model1.Jin,SP_model1.eta_p_max,SP_model1.eta_p_zero);
    title_string = "test spines";
    lab =  'AMPA-R'
    x_label = r'Dendritic distance in ($\mu$M)';
    y_label= r'Protein number';
    folder= "Figures/OneProtein/WithCappedUptake/";
    file_name = folder+"SingleProtein_SingleSim__CappedSpineUptake{0}".format(sim_id);
#     pwa = PlottingWidgetAMPA()
#     pwa.CreateFolderRecursive(folder)
#     PlotSingleSimSingleProtein(x, p_dist, lab, x_label, y_label, title_string, file_name)
    
#     title_string = "Steady-state spatial distribution \n parameters: $D_p = {%.2f}, V_p = {%.1e}$, half-life = %.2f, Jin= %.2f, $\eta_{pmax}$ =%.1e,$\eta_{p0}$ =%.1e" \
#     %( SP_model1.D_p, SP_model1.V_p, SP_model1.half_life,SP_model1.Jin,SP_model1.eta_p_max,SP_model1.eta_p_zero);
    lab_p_spine =  'Spine-AMPA-R'
    x_label = r'Dendritic distance in ($\mu$M)';
    y_label= r'Protein number';
    file_name = folder+"Spine_SingleSim_OneProtein_CappedUptake_dist_{0}".format(sim_id);
    p_spine = SP_model1.eta_p_max*np.tanh(SP_model1.eta_p_zero*p_dist);
    PlotSingleSimSingleProtein(x, p_dist/p_dist[0],p_spine/p_spine[0], lab,lab_p_spine, x_label, y_label, title_string, file_name,fsize=14,save_it = 0)
_ = widgets.interact(RunSim3,v = (0,50,0.1), Jin = (4000,50000,10000), eta_max = (10,60,5))

interactive(children=(FloatSlider(value=0.1, description='v', max=50.0), IntSlider(value=4000, description='Ji…

In [90]:
print(1/2.67)

0.37453183520599254
